In [1]:
import pandas as pd
import os
import numpy as np
import sklearn as scikit
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import altair as alt
from sklearn.decomposition import PCA
from icecream import ic


# Ideas copied from 'Visualizing the embeddings in 2D'

# Sheesh , I'v ebeen losing my EC2 instance as jupyter has been hogging the CPU... use cpulimit

# brew install cpulimit
# cpulimit -l 90 jupyter lab

In [18]:
# Clean data 

In [2]:
df = pd.read_pickle(os.path.expanduser("~/gits/igor2/archive/2022-12-18.journal.gz"))

# Fix up columns
df.date = pd.DatetimeIndex(df.date)
df['embeddings'] = df.embeddings.apply(lambda e: np.array(e))

# throw away short bodies
df = df[df.apply(lambda r: len(r.body)> 500, axis=1)]

# Blind Free Text of Body, can unblind it later if required. 
df['body']=''

# df = df.set_index('date')
df['body']='data blinded'

ic(df.embeddings)
matrix = np.array(df.embeddings.to_list())

ic| df.embeddings: 0       [0.003288468811661005, 0.0104126101359725, 0.0...
                   1       [-0.005472051911056042, -0.002248083706945181,...
                   2       [0.0034192597959190607, -0.003644188167527318,...
                   3       [-0.016969453543424606, -0.0029161879792809486...
                   4       [0.0019290543859824538, -0.019463708624243736,...
                                                 ...                        
                   2446    [-0.03849697858095169, -0.01014410424977541, 0...
                   2447    [-0.01854093186557293, -0.011478052474558353, ...
                   2448    [-0.012807096354663372, -0.010143166407942772,...
                   2449    [-0.010827340185642242, -0.005403110291808844,...
                   2450    [-0.017977548763155937, 0.003252423834055662, ...
                   Name: embeddings, Length: 2414, dtype: object


# Clustering 

In [3]:
# USE PCA to reduce dimensionality
# Consider the other tool from 2D visualization)
df["pca_3d"] = PCA(n_components=3).fit_transform(matrix).tolist()
df["pca_2d"] = PCA(n_components=2).fit_transform(matrix).tolist()

In [4]:
def make_tsne(matrix, n):
    # Create a t-SNE model and transform the data
    tsne = TSNE(
        n_components=n, perplexity=15, random_state=42, init="random", learning_rate=200
    )
    vis_dims = tsne.fit_transform(matrix)
    return vis_dims


tsne = make_tsne(matrix, 2)

In [5]:
df["tsne_2d"] = tsne.tolist()

In [6]:
# https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters/notebook

from sklearn.cluster import MiniBatchKMeans,KMeans
kmeans = KMeans(n_clusters=5).fit(matrix)
df["cluster"]  = kmeans.predict(matrix)

In [7]:
df_graph = pd.DataFrame() 
df_graph["x"] = df.tsne_2d.apply(lambda x: x[0])
df_graph["y"] = df.tsne_2d.apply(lambda x: x[1])
df_graph["color"] = df.date.dt.year
#  df_graph["color"] = df.Family
df_graph["tooltip"] = df.date.dt.to_period("D").apply(str)

import altair as alt
from vega_datasets import data

color_ = alt.Color("color:N")
tooltip="tooltip"
domain_ = ["","Amelia","Zach ", "Zach Amelia"]
range_ = ['grey', 'red', 'green', 'blue']
# color_ = alt.Color('color', scale=alt.Scale(domain=domain_, range=range_))

#sample = df
# sampe = df[:500]
alt.Chart(df_graph).mark_point().encode(
    x="x", y="y", 
    color=color_,
    tooltip="tooltip"
).properties(width=800, height=800).interactive()

alt.Chart(...)

# Finding similar to this day

In [8]:
# Lets try finding items that are similar by distance
# OpenAI recommends cosine similiarity
# https://beta.openai.com/docs/guides/embeddings/which-distance-function-should-i-use

import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
distances = pairwise_distances(matrix, metric='cosine')
def df_from_distances(distances):
    column_names = [f"D{i}" for i in range(len(distances))]
    return pd.DataFrame(distances, columns=column_names)
df_from_distances(distances)
df2 = pd.concat([df, df_from_distances(distances)], axis=1) 

In [9]:
# Search by within percentile range.
# Not should use local percentile, not global percentile
target_percentiles = [i for i in range(99)]
global_p_distances = dict(zip(target_percentiles, np.percentile(distances,target_percentiles) ))

In [10]:
row=34

print (f"Show the closest elements to {df2.iloc[row].date}")
distance_column = f"D{row}"
df2.sort_values(by=distance_column,ascending=True)[:20][["date", distance_column]]

Show the closest elements to 2022-10-31 00:00:00


,date,D34
34,2022-11-02,0.000000
617,2019-10-14,0.093874
583,2019-11-28,0.096089
554,2019-12-31,0.100632
126,2022-06-02,0.100737
589,2019-11-22,0.102089
400,2020-09-19,0.102627
10,2022-12-06,0.103187
646,2019-08-13,0.103739
4,2022-12-14,0.104639


In [11]:
def family(s):
    s = s.lower()
    family = ""
    if "zach" in s:
        family+="Zach " 
    if "amelia" in s:
        family+="Amelia" 
    return family
df["Zach"] = df.body.apply(lambda x:"zach" in x.lower())
df["Amelia"] = df.body.apply(lambda x:"amelia" in x.lower())
df["Family"] = df.body.apply(family)

In [12]:
df.Zach

0       False
1       False
2       False
3       False
4       False
        ...  
2446    False
2447    False
2448    False
2449    False
2450    False
Name: Zach, Length: 2414, dtype: bool

# Use semantic search to find nearest elements with live embeddings

In [13]:
import openai
import json
def setup_gpt():
    PASSWORD = "replaced_from_secret_box"
    with open(os.path.expanduser("~/gits/igor2/secretBox.json")) as json_data:
        SECRETS = json.load(json_data)
        PASSWORD = SECRETS["openai"]
    openai.api_key = PASSWORD
    return openai

import functools

@functools.lru_cache()
def get_embedding(text, model="text-embedding-ada-002"):
    ic ("Calling Online")
    text = text.replace("\n", " ")
    return setup_gpt().Embedding.create(input=[text], model=model)["data"][0]["embedding"]
    # return "hello"


In [15]:
from sklearn.metrics.pairwise import cosine_similarity # (X, Y=None, dense_output=True)

def print_distance(content):
    embedding = np.array(get_embedding(content)).reshape(1,-1)
                                   
    distance_column = "distance"
    distance_column_rebased  = distance_column+"_rebased"

    df[distance_column] = df.embeddings.apply(lambda r: 100*(1- cosine_similarity(r.reshape(1,-1), embedding )[0][0]))
    df[distance_column_rebased] = df[distance_column] - min(df[distance_column])
    metrics = [1, 5,25,50,75,99,100]
    values  = np.percentile( df[distance_column], metrics)

    percentiles = dict(zip(metrics, values))
    percentiles_distance = dict(zip(metrics, values-values[0]))

    ic(percentiles) # Take a quick peek at the percentiles to see the spread (perhaps I should be looking at STDDEV?
    ic(percentiles_distance)

    display(df.sort_values(by=distance_column,ascending=True)[:20][["date", distance_column, distance_column_rebased]])
    return df
                                       
_ = print_distance("Zach breaking his arm")                                

ic| percentiles: {1: 21.79440590326618,
                  5: 23.397285898803236,
                  25: 25.774521611522204,
                  50: 27.00522107847057,
                  75: 27.973851910375018,
                  99: 30.41186610749484,
                  100: 32.52421419136113}
ic| percentiles_distance: {1: 0.0,
                           5: 1.6028799955370552,
                           25: 3.9801157082560223,
                           50: 5.210815175204388,
                           75: 6.179446007108837,
                           99: 8.61746020422866,
                           100: 10.72980828809495}


,date,distance,distance_rebased
487,2020-03-12,18.444729,0.000000
1890,2013-08-19,18.733100,0.288372
76,2022-08-26,19.666320,1.221592
1663,2014-05-02,19.998874,1.554145
50,2022-10-06,20.176286,1.731557
1671,2014-04-24,20.448762,2.004034
1571,2014-08-14,20.487088,2.042359
1767,2014-01-02,20.651617,2.206888
1917,2013-07-22,20.684728,2.239999
1440,2015-01-10,20.754149,2.309420


# Looking for "average days", and outliers ..

In [16]:
# np.shape(matrix)
mean = np.mean(matrix,axis=0)
median = np.percentile(matrix,50,axis=0)

In [ ]:

# sum(median-mean)
ic(sum(mean))
ic(sum(median))
ic(sum(mean-median))

In [ ]:
df["remain"] = df.embeddings.apply(lambda e: np.abs(e - mean))

In [ ]:
df["sum_remain"] = df.remain.apply(np.sum)

In [ ]:
df.sort_values(by="sum_remain",ascending=True)[400:500]